# Results Tables

* **Error Metrics**: RMSE, MAE, MAAO, MAPE, RMSLE
* **Goodness of Fit**: NLL, PR coverage
* **Uncertainty**: PR area, confidence interval (and width)

In [1]:
# import modules
import numpy as np
import pickle
import pandas as pd
import scipy.stats as stats
import scipy.special
from scipy.stats import multivariate_normal
from spatial_plotting import rmse,mae,maao,mape,rmsle
from hypothesis_test_functions import generate_samples,generate_rmsle_samples,get_dists_and_data

In [2]:
# initialisation (indexing for model retrieval)
num_reps = 100
config_ids = [36,37,38,39] # configuration ids of the optimal hyperparameter models
replication_ids = {}
failed_experiment_IDs = []

for index, config_id in enumerate(config_ids):
    experimentIDs = np.arange(index*num_reps+1,(index+1)*num_reps+1) # corresponding experiment IDs 
    # removing experiments that didn't run successfully
    experimentIDs = np.delete(experimentIDs,failed_experiment_IDs)
    replication_ids[config_id] = experimentIDs

In [3]:
# load appropriate data and obtain samples

# initialisation
date = '2025-02-04'
early_stopping = 100
root_dir = r"C:\Users\vm2218\OneDrive - Imperial College London\PhD Project\seaducks\experiments\hpc_runs\04-02-2025\model_test_data"
root_dir_model = r"C:\Users\vm2218\OneDrive - Imperial College London\PhD Project\seaducks\experiments\hpc_runs\04-02-2025\fit_models"
file_name_prefix = "long_experiment_"
file_name_suffix = f"_date_{date}_early_stopping_{early_stopping}"

In [4]:
SST_true_polar_false_config_ID = 38
SST_false_polar_false_config_ID = 36
SST_true_polar_true_config_ID = 39
SST_false_polar_true_config_ID = 37
invalid_vals = []

## Error Metrics

In [5]:
metrics = {'RMSE':rmse,'MAE':mae,'MAAO':maao,'MAPE':mape, 'RMSLE':rmsle}
names = ['SST: True | Polar: False','SST: False | Polar: False','SST: True | Polar: True','SST: False | Polar: True']
df_index = []
for ii,name in enumerate(metrics.keys()):
    df_index+=[name, f"{name} sem", f"{name} 95 percent ci"]

df = pd.DataFrame(dict.fromkeys(names,list(np.zeros(3*len(metrics.keys())))),index=df_index)

for jj,metric_name in enumerate(metrics.keys()):

    print(f'\n .*+ --------------- {metric_name} --------------- +*.')
    metric = metrics[metric_name]

    if metric_name == 'RMSLE':
        X_SST_true_polar_false, X_SST_false_polar_false = generate_rmsle_samples(metric,SST_true_polar_false_config_ID,SST_false_polar_false_config_ID,
                                   num_reps, root_dir,file_name_prefix,replication_ids,file_name_suffix,invalid_vals=invalid_vals)
        X_SST_true_polar_true, X_SST_false_polar_true = generate_rmsle_samples(metric,SST_true_polar_true_config_ID,SST_false_polar_true_config_ID,
                                   num_reps, root_dir,file_name_prefix,replication_ids,file_name_suffix,invalid_vals=invalid_vals)
    else:
         X_SST_true_polar_false, X_SST_false_polar_false = generate_samples(metric,SST_true_polar_false_config_ID,SST_false_polar_false_config_ID,
                                   num_reps, root_dir,file_name_prefix,replication_ids,file_name_suffix,invalid_vals=invalid_vals)
         X_SST_true_polar_true, X_SST_false_polar_true = generate_samples(metric,SST_true_polar_true_config_ID,SST_false_polar_true_config_ID,
                                   num_reps, root_dir,file_name_prefix,replication_ids,file_name_suffix,invalid_vals=invalid_vals)

    sem_SST_true_polar_false = stats.sem(X_SST_true_polar_false)
    sem_SST_false_polar_false = stats.sem(X_SST_false_polar_false)
    sem_SST_true_polar_true = stats.sem(X_SST_true_polar_true)
    sem_SST_false_polar_true = stats.sem(X_SST_false_polar_true)

    sems = [sem_SST_true_polar_false, sem_SST_false_polar_false, sem_SST_true_polar_true, sem_SST_false_polar_true]
    samples = [X_SST_true_polar_false, X_SST_false_polar_false,X_SST_true_polar_true, X_SST_false_polar_true]

    for ii in range(4):
        
        print(f"\n{names[ii]}\n{"_"*len(names[ii])}")
        print(f"{metric_name}: {np.mean(samples[ii]):.3f} +- {sems[ii]:.3f}")
        t_interval = stats.t.interval(confidence=0.95, df=len(samples[ii])-1, 
              loc=np.mean(samples[ii]), 
              scale=sems[ii])
        print(f"95 percent confidence interval for the mean of {metric_name}: {[round(val,2) for val in t_interval]}")
        print(f"    width: {t_interval[1]-t_interval[0]:.3f}")
        
        df.iloc[3*jj,ii] = np.mean(samples[ii])
        df.iloc[3*jj+1,ii] = sems[ii]
        df.iloc[3*jj+2,ii] = str(tuple(map(lambda x:round(x,2),t_interval)))


 .*+ --------------- RMSE --------------- +*.

SST: True | Polar: False
________________________
RMSE: 14.543 +- 0.045
95 percent confidence interval for the mean of RMSE: [14.45, 14.63]
    width: 0.180

SST: False | Polar: False
_________________________
RMSE: 14.784 +- 0.046
95 percent confidence interval for the mean of RMSE: [14.69, 14.88]
    width: 0.182

SST: True | Polar: True
_______________________
RMSE: 14.534 +- 0.046
95 percent confidence interval for the mean of RMSE: [14.44, 14.62]
    width: 0.181

SST: False | Polar: True
________________________
RMSE: 14.750 +- 0.046
95 percent confidence interval for the mean of RMSE: [14.66, 14.84]
    width: 0.183

 .*+ --------------- MAE --------------- +*.


C:\Users\vm2218\AppData\Local\Temp\ipykernel_900\3602386472.py:45: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '(14.45, 14.63)' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.iloc[3*jj+2,ii] = str(tuple(map(lambda x:round(x,2),t_interval)))
C:\Users\vm2218\AppData\Local\Temp\ipykernel_900\3602386472.py:45: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '(14.69, 14.88)' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.iloc[3*jj+2,ii] = str(tuple(map(lambda x:round(x,2),t_interval)))
C:\Users\vm2218\AppData\Local\Temp\ipykernel_900\3602386472.py:45: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '(14.44, 14.62)' has dtype incompatible with float64, please 


SST: True | Polar: False
________________________
MAE: 9.844 +- 0.030
95 percent confidence interval for the mean of MAE: [9.78, 9.9]
    width: 0.119

SST: False | Polar: False
_________________________
MAE: 9.964 +- 0.030
95 percent confidence interval for the mean of MAE: [9.9, 10.02]
    width: 0.120

SST: True | Polar: True
_______________________
MAE: 9.857 +- 0.030
95 percent confidence interval for the mean of MAE: [9.8, 9.92]
    width: 0.120

SST: False | Polar: True
________________________
MAE: 9.960 +- 0.030
95 percent confidence interval for the mean of MAE: [9.9, 10.02]
    width: 0.120

 .*+ --------------- MAAO --------------- +*.

SST: True | Polar: False
________________________
MAAO: 40.079 +- 0.072
95 percent confidence interval for the mean of MAAO: [39.94, 40.22]
    width: 0.284

SST: False | Polar: False
_________________________
MAAO: 40.485 +- 0.071
95 percent confidence interval for the mean of MAAO: [40.34, 40.63]
    width: 0.281

SST: True | Polar: True


In [6]:
latex_df = df.copy()
N,p = df.shape
n_round = 2
def pad_num(x):
    out = str(x)
    ind = out.index('.')
    out = out+(n_round-(len(out)-(ind+1)))*"0"
    return out

for ii in range(4):
    for jj,metric_name in enumerate(metrics.keys()):
        _str = f"{pad_num(np.round(df.iloc[3*jj,ii],2))} ± {pad_num(np.round(df.iloc[3*jj+1,ii], 2))}" 
        latex_df.iloc[3*jj,ii] = _str
        
latex_df.drop(latex_df.index[[3*jj+1 for jj in range(len(metrics.keys()))]])
        

,SST: True | Polar: False,SST: False | Polar: False,SST: True | Polar: True,SST: False | Polar: True
RMSE,14.54 ± 0.05,14.78 ± 0.05,14.53 ± 0.05,14.75 ± 0.05
RMSE 95 percent ci,"(14.45, 14.63)","(14.69, 14.88)","(14.44, 14.62)","(14.66, 14.84)"
MAE,9.84 ± 0.03,9.96 ± 0.03,9.86 ± 0.03,9.96 ± 0.03
MAE 95 percent ci,"(9.78, 9.9)","(9.9, 10.02)","(9.8, 9.92)","(9.9, 10.02)"
MAAO,40.08 ± 0.07,40.49 ± 0.07,40.12 ± 0.07,40.49 ± 0.07
MAAO 95 percent ci,"(39.94, 40.22)","(40.34, 40.63)","(39.97, 40.26)","(40.35, 40.63)"
MAPE,439.77 ± 8.83,442.71 ± 9.03,441.26 ± 8.81,449.98 ± 9.38
MAPE 95 percent ci,"(422.25, 457.29)","(424.8, 460.63)","(423.78, 458.75)","(431.37, 468.6)"
RMSLE,0.22 ± 0.00,0.23 ± 0.00,0.22 ± 0.00,0.23 ± 0.00
RMSLE 95 percent ci,"(0.22, 0.22)","(0.22, 0.23)","(0.22, 0.22)","(0.22, 0.23)"


In [7]:
filehandler = open(f"error_metric_values.p","wb")
pickle.dump(latex_df,filehandler)

## Goodness of Fit

In [8]:
dists_SST_true_polar_false, y_SST_true_polar_false, dists_SST_false_polar_false, y_SST_false_polar_false = get_dists_and_data(SST_true_polar_false_config_ID,SST_false_polar_false_config_ID,
                                   num_reps, root_dir,root_dir_model,file_name_prefix,replication_ids,file_name_suffix,invalid_vals=invalid_vals)

dists_SST_true_polar_true, y_SST_true_polar_true, dists_SST_false_polar_true, y_SST_false_polar_true = get_dists_and_data(SST_true_polar_true_config_ID,SST_false_polar_true_config_ID,
                                   num_reps, root_dir,root_dir_model,file_name_prefix,replication_ids,file_name_suffix,invalid_vals=invalid_vals)

In [9]:
scipy_dists_SST_true_polar_false = [multivariate_normal(mean=dists_SST_true_polar_false[0][0][ii],cov=dists_SST_true_polar_false[0][1][ii]) for ii in range(dists_SST_true_polar_false[0][0].shape[0])]
scipy_dists_SST_false_polar_false = [multivariate_normal(mean= dists_SST_false_polar_false[0][0][ii],cov= dists_SST_false_polar_false[0][1][ii]) for ii in range( dists_SST_false_polar_false[0][0].shape[0])]
scipy_dists_SST_true_polar_true = [multivariate_normal(mean=dists_SST_true_polar_true[0][0][ii],cov=dists_SST_true_polar_true[0][1][ii]) for ii in range(dists_SST_true_polar_true[0][0].shape[0])]
scipy_dists_SST_false_polar_true = [multivariate_normal(mean=dists_SST_false_polar_true[0][0][ii],cov=dists_SST_false_polar_true[0][1][ii]) for ii in range(dists_SST_false_polar_true[0][0].shape[0])]

In [10]:
def NLL(dists, y):
    N = y[0].shape[0]
    nlls = [-dists[i].logpdf(y[0][i, :]) for i in range(N)]
    return nlls

def in_ci(error, sigma, alpha=0.95):
    crit_val = stats.chi2.ppf(alpha, sigma.shape[0])
    sigma_inv = np.linalg.inv(sigma)
    n = error.T @ sigma_inv @ error
    return n < crit_val

def coverage(dists, y, alpha=0.90):
    N = y[0].shape[0]
    covs = [dists[i].cov for i in range(N)]
    diffs = [y[0][i, :] - dists[i].mean for i in range(N)]
    ret = [in_ci(diff, cov, alpha=alpha) for diff, cov in zip(diffs, covs)]
    ret = sum(ret)/(len(ret))
    return ret

def matrix_area(mat, mult):
    p = mat.shape[0]
    return (
        2
        * (np.pi ** (p / 2))
        / scipy.special.gamma(p / 2)
        / p
        * (np.linalg.det(mat) ** 0.5)
        * mult ** (p / 2)
    )

def area(dists, y, alpha: float = 0.90):
    N = y[0].shape[0]
    covs = [dists[i].cov for i in range(N)]
    # TODO: Check this is correct for the area
    area = [matrix_area(cov, stats.chi2.ppf(alpha, cov.shape[1]))*100**2 for cov in covs] #check that putting chi2 here was the correct thing to do
    return area

In [11]:
goodness_of_fit_metrics = {'NLL':NLL, 'PR area':area, 'PR coverage':coverage}
df_index = []

for ii,name in enumerate(goodness_of_fit_metrics.keys()):
    df_index+=[name, f"{name} sem"]

df = pd.DataFrame(dict.fromkeys(names,list(np.zeros(2*len(goodness_of_fit_metrics.keys())))),index=df_index)

for jj,metric_name in enumerate(goodness_of_fit_metrics.keys()):

    print(f'\n .*+ --------------- {metric_name} --------------- +*.')
    metric = goodness_of_fit_metrics[metric_name]

    SST_true_polar_false_name = 'SST: True | Polar: False'
    SST_false_polar_false_name = 'SST: False | Polar: False'
    SST_true_polar_true_name = 'SST: True | Polar: True'
    SST_false_polar_true_name = 'SST: False | Polar: True'
    
    dists_SST_true_polar_false = metric(scipy_dists_SST_true_polar_false,y_SST_true_polar_false)
    dists_SST_false_polar_false = metric(scipy_dists_SST_false_polar_false,y_SST_false_polar_false)
    dists_SST_true_polar_true = metric(scipy_dists_SST_true_polar_true,y_SST_true_polar_true)
    dists_SST_false_polar_true = metric(scipy_dists_SST_false_polar_true,y_SST_false_polar_true)

    if metric_name != 'PR coverage':
        sem_SST_true_polar_false = stats.sem(dists_SST_true_polar_false)
        sem_SST_false_polar_false = stats.sem(dists_SST_false_polar_false)
        sem_SST_true_polar_true = stats.sem(dists_SST_true_polar_true)
        sem_SST_false_polar_true = stats.sem(dists_SST_false_polar_true)

        sems = [sem_SST_true_polar_false, sem_SST_false_polar_false, sem_SST_true_polar_true, sem_SST_false_polar_true]
    names = [SST_true_polar_false_name,SST_false_polar_false_name,SST_true_polar_true_name,SST_false_polar_true_name]
    samples = [dists_SST_true_polar_false, dists_SST_false_polar_false,dists_SST_true_polar_true, dists_SST_false_polar_true]

    for ii in range(4):
        print(f"\n{names[ii]}\n{"_"*len(names[ii])}")
        if metric_name == 'PR area':
            print(f"{metric_name}: {round(np.mean(samples[ii]))} +- {round(sems[ii])}")
        elif metric_name == 'PR coverage':
            print(f"{metric_name}: {np.mean(samples[ii]):.3f}")
        else:
            print(f"{metric_name}: {np.mean(samples[ii]):.3f} +- {sems[ii]:.3f}")
        
        df.iloc[2*jj,ii] = np.mean(samples[ii])
        df.iloc[2*jj+1,ii] = sems[ii]




 .*+ --------------- NLL --------------- +*.

SST: True | Polar: False
________________________
NLL: -1.465 +- 0.009

SST: False | Polar: False
_________________________
NLL: -1.441 +- 0.010

SST: True | Polar: True
_______________________
NLL: -1.456 +- 0.011

SST: False | Polar: True
________________________
NLL: -1.426 +- 0.010

 .*+ --------------- PR area --------------- +*.

SST: True | Polar: False
________________________
PR area: 2252 +- 11

SST: False | Polar: False
_________________________
PR area: 2396 +- 13

SST: True | Polar: True
_______________________
PR area: 2262 +- 12

SST: False | Polar: True
________________________
PR area: 2390 +- 13

 .*+ --------------- PR coverage --------------- +*.

SST: True | Polar: False
________________________
PR coverage: 0.862

SST: False | Polar: False
_________________________
PR coverage: 0.863

SST: True | Polar: True
_______________________
PR coverage: 0.860

SST: False | Polar: True
________________________
PR coverage: 0.86

In [12]:
latex_df = df.copy()
N,p = df.shape
n_round = 2
def pad_num(x):
    out = str(x)
    ind = out.index('.')
    out = out+(n_round-(len(out)-(ind+1)))*"0"
    return out

for ii in range(4):
    for jj,metric_name in enumerate(goodness_of_fit_metrics.keys()):
        if metric_name == 'PR coverage':
            _str = f"{pad_num(np.round(df.iloc[2*jj,ii],2))} ± {pad_num(0.00)}" 
            latex_df.iloc[2*jj,ii] = _str
        elif metric_name == 'PR area':
            _str = f"{round(df.iloc[2*jj,ii])} ± {round(df.iloc[2*jj+1,ii])}" 
            latex_df.iloc[2*jj,ii] = _str
        else:
            _str = f"{pad_num(np.round(df.iloc[2*jj,ii],2))} ± {pad_num(np.round(df.iloc[2*jj+1,ii], 2))}" 
            latex_df.iloc[2*jj,ii] = _str

latex_df.drop(latex_df.index[[2*jj+1 for jj in range(len(goodness_of_fit_metrics.keys()))]])

        

C:\Users\vm2218\AppData\Local\Temp\ipykernel_900\1788703537.py:20: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '-1.46 ± 0.01' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  latex_df.iloc[2*jj,ii] = _str
C:\Users\vm2218\AppData\Local\Temp\ipykernel_900\1788703537.py:20: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '-1.44 ± 0.01' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  latex_df.iloc[2*jj,ii] = _str
C:\Users\vm2218\AppData\Local\Temp\ipykernel_900\1788703537.py:20: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '-1.46 ± 0.01' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  latex_df.iloc[2*jj,ii] = _str
C:\User

,SST: True | Polar: False,SST: False | Polar: False,SST: True | Polar: True,SST: False | Polar: True
NLL,-1.46 ± 0.01,-1.44 ± 0.01,-1.46 ± 0.01,-1.43 ± 0.01
PR area,2252 ± 11,2396 ± 13,2262 ± 12,2390 ± 13
PR coverage,0.86 ± 0.00,0.86 ± 0.00,0.86 ± 0.00,0.86 ± 0.00


In [13]:
filehandler = open(f"goodness_of_fit_metric_values.p","wb")
pickle.dump(latex_df,filehandler)

## Feature Importances

In [14]:
# load models
models = {}
for config_ID in config_ids:
    with open(fr'{root_dir_model}/{file_name_prefix}{replication_ids[config_ID][ii]}{file_name_suffix}.p', 'rb') as pickle_file:
                models[config_ID]= pickle.load(pickle_file)
                

In [15]:
def pad_num(x):
    out = str(x)
    ind = out.index('.')
    out = out+(n_round-(len(out)-(ind+1)))*"0"
    return out

SST_true_polar_false_name = 'SST: True | Polar: False'
SST_false_polar_false_name = 'SST: False | Polar: False'
SST_true_polar_true_name = 'SST: True | Polar: True'
SST_false_polar_true_name = 'SST: False | Polar: True'

names = [SST_true_polar_false_name,SST_false_polar_false_name,SST_true_polar_true_name,SST_false_polar_true_name]
params = ['mu_x', 'mu_y', 'a_11', 'a_12','a_22']
explanatory_var_labels = ['u_av','v_av','lat','lon','day_of_year','Wx','Wy','Tx','Ty','sst_x_derivative','sst_y_derivative']
config_ids = [SST_true_polar_false_config_ID,SST_false_polar_false_config_ID,SST_true_polar_true_config_ID,SST_false_polar_true_config_ID]

print(f'\n .*+ --------------- Feature Importance --------------- +*.')
for kk in range(4):
    df = pd.DataFrame(dict.fromkeys(params,list(np.zeros(len(explanatory_var_labels)))),index=explanatory_var_labels)

    print(f"\n{names[kk]}\n{"_"*len(names[kk])}")
    print("\nFeature Importances (Gini Index):")
    fi = models[config_ids[kk]].feature_importances_
    if kk%2 == 1:
        for jj,param in enumerate(params):
            print(f'\nFeature Importances for {param}')
            for ii, var in enumerate(explanatory_var_labels[:9]):
                print(f"{var}: {fi[jj][ii]:.2f}")
                df.iloc[ii,jj] = fi[jj][ii]
            
                
    else:
        for jj,param in enumerate(params):
            print(f'\nFeature Importances for {param}')
            for ii, var in enumerate(explanatory_var_labels):
                print(f"{var}: {fi[jj][ii]:.2f}")

                df.iloc[ii,jj] = fi[jj][ii]

    latex_df = df.copy()
    N,p = df.shape
    n_round = 2

    for jj in range(len(params)):
        for ii,var in enumerate(explanatory_var_labels):
            _str = f"{pad_num(np.round(df.iloc[ii,jj],2))}" 
            latex_df.iloc[ii,jj] = _str
    
    # putting numbers in tables
    filehandler = open(f"model_{kk}_feature_importances_values.p","wb")
    pickle.dump(latex_df,filehandler)



 .*+ --------------- Feature Importance --------------- +*.

SST: True | Polar: False
________________________

Feature Importances (Gini Index):

Feature Importances for mu_x
u_av: 0.11
v_av: 0.07
lat: 0.27
lon: 0.24
day_of_year: 0.06
Wx: 0.03
Wy: 0.03
Tx: 0.03
Ty: 0.03
sst_x_derivative: 0.06
sst_y_derivative: 0.07

Feature Importances for mu_y
u_av: 0.07
v_av: 0.13
lat: 0.24
lon: 0.22
day_of_year: 0.07
Wx: 0.03
Wy: 0.03
Tx: 0.04
Ty: 0.03
sst_x_derivative: 0.08
sst_y_derivative: 0.06

Feature Importances for a_11
u_av: 0.10
v_av: 0.08
lat: 0.20
lon: 0.17
day_of_year: 0.10
Wx: 0.05
Wy: 0.05
Tx: 0.05
Ty: 0.05
sst_x_derivative: 0.07
sst_y_derivative: 0.07

Feature Importances for a_12
u_av: 0.09
v_av: 0.10
lat: 0.17
lon: 0.15
day_of_year: 0.08
Wx: 0.07
Wy: 0.05
Tx: 0.06
Ty: 0.06
sst_x_derivative: 0.08
sst_y_derivative: 0.09

Feature Importances for a_22
u_av: 0.09
v_av: 0.10
lat: 0.20
lon: 0.18
day_of_year: 0.10
Wx: 0.05
Wy: 0.05
Tx: 0.05
Ty: 0.04
sst_x_derivative: 0.07
sst_y_derivative

C:\Users\vm2218\AppData\Local\Temp\ipykernel_900\2782300957.py:47: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.11' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  latex_df.iloc[ii,jj] = _str
C:\Users\vm2218\AppData\Local\Temp\ipykernel_900\2782300957.py:47: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.07' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  latex_df.iloc[ii,jj] = _str
C:\Users\vm2218\AppData\Local\Temp\ipykernel_900\2782300957.py:47: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.10' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  latex_df.iloc[ii,jj] = _str
C:\Users\vm2218\AppData\Local\Temp\ip


Feature Importances for mu_x
u_av: 0.12
v_av: 0.07
lat: 0.29
lon: 0.28
day_of_year: 0.07
Wx: 0.04
Wy: 0.04
Tx: 0.04
Ty: 0.05

Feature Importances for mu_y
u_av: 0.10
v_av: 0.14
lat: 0.25
lon: 0.24
day_of_year: 0.09
Wx: 0.05
Wy: 0.04
Tx: 0.04
Ty: 0.04

Feature Importances for a_11
u_av: 0.12
v_av: 0.09
lat: 0.23
lon: 0.19
day_of_year: 0.13
Wx: 0.06
Wy: 0.06
Tx: 0.06
Ty: 0.06

Feature Importances for a_12
u_av: 0.10
v_av: 0.10
lat: 0.22
lon: 0.18
day_of_year: 0.11
Wx: 0.08
Wy: 0.06
Tx: 0.07
Ty: 0.07

Feature Importances for a_22
u_av: 0.11
v_av: 0.11
lat: 0.22
lon: 0.21
day_of_year: 0.13
Wx: 0.06
Wy: 0.06
Tx: 0.06
Ty: 0.05

SST: True | Polar: True
_______________________

Feature Importances (Gini Index):

Feature Importances for mu_x
u_av: 0.11
v_av: 0.08
lat: 0.26
lon: 0.23
day_of_year: 0.06
Wx: 0.04
Wy: 0.03
Tx: 0.04
Ty: 0.03
sst_x_derivative: 0.05
sst_y_derivative: 0.07

Feature Importances for mu_y
u_av: 0.13
v_av: 0.09
lat: 0.21
lon: 0.21
day_of_year: 0.07
Wx: 0.04
Wy: 0.03
Tx: 0.

C:\Users\vm2218\AppData\Local\Temp\ipykernel_900\2782300957.py:47: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.11' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  latex_df.iloc[ii,jj] = _str
C:\Users\vm2218\AppData\Local\Temp\ipykernel_900\2782300957.py:47: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.13' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  latex_df.iloc[ii,jj] = _str
C:\Users\vm2218\AppData\Local\Temp\ipykernel_900\2782300957.py:47: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.09' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  latex_df.iloc[ii,jj] = _str
C:\Users\vm2218\AppData\Local\Temp\ip


SST: False | Polar: True
________________________

Feature Importances (Gini Index):

Feature Importances for mu_x
u_av: 0.12
v_av: 0.09
lat: 0.30
lon: 0.27
day_of_year: 0.06
Wx: 0.04
Wy: 0.03
Tx: 0.04
Ty: 0.03

Feature Importances for mu_y
u_av: 0.14
v_av: 0.10
lat: 0.25
lon: 0.24
day_of_year: 0.09
Wx: 0.05
Wy: 0.04
Tx: 0.05
Ty: 0.04

Feature Importances for a_11
u_av: 0.12
v_av: 0.10
lat: 0.24
lon: 0.20
day_of_year: 0.11
Wx: 0.06
Wy: 0.05
Tx: 0.07
Ty: 0.05

Feature Importances for a_12
u_av: 0.13
v_av: 0.12
lat: 0.18
lon: 0.18
day_of_year: 0.10
Wx: 0.07
Wy: 0.07
Tx: 0.08
Ty: 0.08

Feature Importances for a_22
u_av: 0.11
v_av: 0.11
lat: 0.22
lon: 0.20
day_of_year: 0.13
Wx: 0.06
Wy: 0.05
Tx: 0.07
Ty: 0.06


In [16]:

latex_df
        

,mu_x,mu_y,a_11,a_12,a_22
u_av,0.12,0.14,0.12,0.13,0.11
v_av,0.09,0.10,0.10,0.12,0.11
lat,0.30,0.25,0.24,0.18,0.22
lon,0.27,0.24,0.20,0.18,0.20
day_of_year,0.06,0.09,0.11,0.10,0.13
Wx,0.04,0.05,0.06,0.07,0.06
Wy,0.03,0.04,0.05,0.07,0.05
Tx,0.04,0.05,0.07,0.08,0.07
Ty,0.03,0.04,0.05,0.08,0.06
sst_x_derivative,0.00,0.00,0.00,0.00,0.00


In [17]:
latex_df = latex_df.style.set_caption(f"{names[kk]}")
latex_df


,mu_x,mu_y,a_11,a_12,a_22
u_av,0.12,0.14,0.12,0.13,0.11
v_av,0.09,0.10,0.10,0.12,0.11
lat,0.30,0.25,0.24,0.18,0.22
lon,0.27,0.24,0.20,0.18,0.20
day_of_year,0.06,0.09,0.11,0.10,0.13
Wx,0.04,0.05,0.06,0.07,0.06
Wy,0.03,0.04,0.05,0.07,0.05
Tx,0.04,0.05,0.07,0.08,0.07
Ty,0.03,0.04,0.05,0.08,0.06
sst_x_derivative,0.00,0.00,0.00,0.00,0.00
